# WallStreetBets Analysis

In [1]:
import pandas as pd
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath

In [2]:
chunksize=10**3
wsb = pd.read_json('/home/aaruran/Documents/Git/wsbData.json',
                   lines=True, chunksize=chunksize)

In [3]:
corpus = datapath('/home/aaruran/Documents/Git/wsbData.json')

In [4]:
model = FastText(size=100)

In [5]:
model.build_vocab(corpus_file=corpus)

In [ ]:
model.train(corpus_file=corpus, epochs=model.epochs,
           total_examples=model.corpus_count, total_words=model.corpus_total_words)

In [ ]:
model.save('wsb_gensim_model')

In [ ]:
def create_comment_vectors(chunk):
    cleaned_chunk = chunk[['body', 'created_utc']]

    # cleaned_chunk = cleaned_chunk['body]
    cleaned_chunk = cleaned_chunk[cleaned_chunk['body'] != '[deleted]']
    cleaned_chunk = cleaned_chunk[cleaned_chunk['body'] != '[removed]']
    cleaned_chunk['created_utc'] = pd.to_datetime(chunk['created_utc'], unit='s',
                                                  origin='unix')
    cleaned_chunk = cleaned_chunk.sort_values(by=['created_utc'])
    cleaned_chunk['created_utc'] = cleaned_chunk['created_utc'].dt.date
    cleaned_chunk['embeddings'] = model.wv[chunk['body']].tolist()
    
for chunk in wsb:
    cleaned_chunk = create_comment_vectors(chunk)
    cleaned_chunk.to_csv('/home/aaruran/Documents/Git/cleaned_wsb.csv',
                        head=True, mode='a')

In [ ]:
# model.estimate_memory()

In [ ]:
# import spacy
# nlp = spacy.load('en_core_web_sm', disable=["tagger", "parser", 'ner'])


In [ ]:
# nlp.add_pipe(nlp.create_pipe('sentencizer'))